In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import json
import csv
import os

# ---------------- CONFIGURATION ----------------
DB_CONFIG = {
    "user": "root",
    "password": "root",
    "host": "localhost",
    "port": 8889,
    "database": "RECETTE"
}

class RecipeDatabaseLoader:
    def __init__(self, db_config: dict):
        url = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
        self.engine = create_engine(url)

    def load_recipes(self) -> pd.DataFrame:
        # On utilise une sous-requête pour les ingrédients pour ne perdre AUCUNE ligne de recette
        query = """
        SELECT 
            r.*,
            (SELECT JSON_ARRAYAGG(JSON_OBJECT('nom', i.nom))
             FROM recette_ingredients ri
             JOIN ingredients i ON ri.ingredient_id = i.id
             WHERE ri.recette_id = r.id) AS ingredients_json
        FROM recettes r
        """
        print("📥 Extraction des données...")
        return pd.read_sql(query, self.engine)

class RecipeDataCleaner:
    def _clean_text(self, text):
        if pd.isna(text) or str(text).lower() == 'none': return ""
        return str(text).strip()

    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()

        # 1. RÉPARATION DU DÉCALAGE (Temps dans Difficulté)
        # Si 'difficulte' contient "min" ou est un chiffre pur, on le bascule dans temps_preparation
        def fix_mapping(row):
            diff = str(row['difficulte']).lower()
            if 'min' in diff or 'h' in diff or diff.isdigit():
                # On essaie d'extraire le nombre pour le temps
                minutes = re.findall(r'\d+', diff)
                if minutes:
                    row['temps_preparation'] = int(minutes[0])
                row['difficulte'] = "Non précisée"
            return row

        df = df.apply(fix_mapping, axis=1)

        # 2. PARSING DES INGRÉDIENTS
        def parse_ing(x):
            if not x: return []
            try:
                data = json.loads(x) if isinstance(x, str) else x
                return [i for i in data if i.get('nom')] # On garde que les noms valides
            except: return []

        df['ingredients'] = df['ingredients_json'].apply(parse_ing)
        
        # 3. NETTOYAGE GÉNÉRAL
        for col in ['titre', 'description', 'cuisine', 'difficulte']:
            df[col] = df[col].apply(self._clean_text)
            
        return df

    def assign_metadata(self, df: pd.DataFrame) -> pd.DataFrame:
        # Catégories simplifiées
        def get_cat(t):
            t = t.lower()
            if any(x in t for x in ['entrée', 'salade', 'soupe']): return 'Entrée'
            if any(x in t for x in ['dessert', 'gâteau', 'crème']): return 'Dessert'
            return 'Plat'
        
        df['categorie'] = df['titre'].apply(get_cat)
        # Pays par défaut
        df['pays'] = df['cuisine'].apply(lambda x: x if x else "Autre")
        return df

# ---------------- EXÉCUTION ----------------
if __name__ == "__main__":
    loader = RecipeDatabaseLoader(DB_CONFIG)
    cleaner = RecipeDataCleaner()

    # Flux de données
    df_raw = loader.load_recipes()
    df_clean = cleaner.clean(df_raw)
    df_clean = cleaner.assign_metadata(df_clean)

    # Sélection des colonnes finales
    cols = ['id', 'titre', 'description', 'pays', 'categorie', 'difficulte', 
            'temps_preparation', 'temps_cuisson', 'vegetarien', 'ingredients']
    
    # SAUVEGARDE JSON 
    df_clean[cols].to_json("recettes_finales.json", orient="records", force_ascii=False, indent=4)

    # SAUVEGARDE CSV 
    df_csv = df_clean[cols].copy()
    df_csv['ingredients'] = df_csv['ingredients'].apply(lambda x: ", ".join([i['nom'] for i in x]))
    df_csv.to_csv("recettes_finales.csv", index=False, encoding="utf-8-sig")

    print(f"✅ Analyse terminée : {len(df_clean)} recettes traitées.")

📥 Extraction des données...
✅ Analyse terminée : 807 recettes traitées.
